In [1]:
import pandas as pd
import numpy as np
import yaml
from sklearn.metrics import mean_squared_error
from C_functions_opti import get_LHS_samples, model_optimization, model_sensitivity_analysis, plot_estimation
from SALib.sample import saltelli
from SALib.analyze import sobol

##### 1. Load all data that is needed

In [2]:
# Load experimental data
df_exp = pd.read_csv('data/data_combined.csv')
biomass_exp = df_exp['Biomass [g/L]']
substrate_exp = df_exp['Glucose [g/L]']

In [3]:
# Load parameters from YAML file
with open('config/parameters.yml', 'r') as file:
    param = yaml.safe_load(file)

##### 2. Define all possible equations for mu and qs

The growth rate and the substrate uptake rate are dependend of each other - meaning that if we define a equation for **mu**, then **qs** is defined by mu with $qs=mu/Yxs$. Contrarily, when **qs** is defined by an ODE, then **mu** is calculated with $mu = qs * Yxs$.

In [4]:
mu0 = lambda mu_max, c_glucose, Ks: mu_max * c_glucose / (c_glucose + Ks) # -- MONOD
mu1 = lambda mu_max, c_biomass, X_max: mu_max * (1 - (c_biomass/ X_max)) # -- LOGISTIC
mu2 = lambda mu_max, c_glucose, Ks, c_biomass, X_max: mu_max * (c_glucose / (c_glucose + Ks)) * (1 - (c_biomass/ X_max)) # -- MONOD + LOGISTIC
mu3 = lambda mu_max, c_glucose, Ks, Ki, c_biomass, X_max: mu_max * (c_glucose / (c_glucose + Ks + (c_glucose**2/ Ki))) * (1 - (c_biomass/ X_max)) # -- MONOD + LOGISTIC + INHIBITION
mu4 = lambda qs, Yxs: qs * Yxs # for changing qs

In [5]:
qs0 = lambda qs_max, c_glucose, Ks_qs: qs_max * c_glucose / (Ks_qs + c_glucose) # -- MONOD
qs1 = lambda qs_max, c_glucose, Ks_qs, Ki, glu_met: qs_max * c_glucose / (Ks_qs + c_glucose) * (Ki / (Ki + glu_met)) # -- MONOD + NON COMPETITIVE INHIBITION
qs2 = lambda qs_max, c_glucose, Ks_qs, c_biomass, lag: qs_max * c_glucose / (Ks_qs + c_glucose) * (1 / (np.exp(c_biomass * lag))) # -- MONOD + METABOLIZED GLU
qs3 = lambda mu, Yxs: mu / Yxs # for changing mu

In [6]:
mu_all=[mu0, mu1, mu2, mu3, mu4]
qs_all=[qs0, qs1, qs2, qs3]

##### 3. Define functions that are needed

In [7]:
# Root mean squared error is the objective function
def objective_function(parameters, mu_eq, num_mu, qs_eq, num_qs):
    # Solve the model using the optimal parameters
    time_pred, biomass_pred, substrate_pred, volume_pred = model_optimization(param, parameters, mu_eq, num_mu, qs_eq, num_qs)  # Solve the model using the current parameters
    biomass = pd.concat([biomass_exp, pd.Series(biomass_pred)], axis=1, keys=['biomass_exp', 'biomass_pred']).dropna()
    biomass_exp_ = biomass['biomass_exp'].values
    biomass_pred_ = biomass['biomass_pred'].values
    mse_x = mean_squared_error(biomass_exp_, biomass_pred_)  # Calculate mean squared error for biomass

    glucose = pd.concat([substrate_exp, pd.Series(substrate_pred)], axis=1, keys=['substrate_exp', 'substrate_pred']).dropna()
    substrate_exp_ = glucose['substrate_exp'].values
    substrate_pred_ = glucose['substrate_pred'].values
    mse_s = mean_squared_error(substrate_exp_, substrate_pred_)  # Calculate mean squared error for substrate
    
    # Calculate the combined rmse
    mse = (mse_x + mse_s)/2
    rmse = np.sqrt(mse)  # Calculate root mean squared error
    return rmse, time_pred, biomass_pred, substrate_pred, volume_pred

##### 4. Define set of parameters (3 Versions)

Version 1: Define all parameters manually in the config/parameters.yml file

In [8]:
'''## not needed anymore
# Extract the parameter combination and the number of the set in order to save it in the correct folder
est_mu_max = param['est_mu_max'] # ['set0']
est_Ks = param['est_ks']'''

"## not needed anymore\n# Extract the parameter combination and the number of the set in order to save it in the correct folder\nest_mu_max = param['est_mu_max'] # ['set0']\nest_Ks = param['est_ks']"

Version 2: Apply LH sampling on all parameters

In [9]:
'''# Set the number of samples and parameters
num_samples = 50
num_parameters = 9

# Define the ranges for each parameter
parameter_bounds = [
    [0.01, 0.6],    # Range for parameter 1 mu_max
    [18, 25],       # Range for parameter 2 X_max
    [0.1, 10.0],    # Range for parameter 3 - Ks
    [0.1, 10.0],    # Range for parameter 4 - Ks_qs
    [6.0, 8.0],     # Range for parameter 5 - Ki
    [0.2, 0.6],     # Range for parameter 6 - Yxs
    [0.5, 1.5],     # Range for parameter 7 - qs_max
    [0.05, 0.5],   # Range for parameter 8 - m_s
    [0.001, 0.02],  # Range for parameter 9 - lag
]'''

'# Set the number of samples and parameters\nnum_samples = 50\nnum_parameters = 9\n\n# Define the ranges for each parameter\nparameter_bounds = [\n    [0.01, 0.6],    # Range for parameter 1 mu_max\n    [18, 25],       # Range for parameter 2 X_max\n    [0.1, 10.0],    # Range for parameter 3 - Ks\n    [0.1, 10.0],    # Range for parameter 4 - Ks_qs\n    [6.0, 8.0],     # Range for parameter 5 - Ki\n    [0.2, 0.6],     # Range for parameter 6 - Yxs\n    [0.5, 1.5],     # Range for parameter 7 - qs_max\n    [0.05, 0.5],   # Range for parameter 8 - m_s\n    [0.001, 0.02],  # Range for parameter 9 - lag\n]'

Version 3: Use some defined parameters and some LH samples

In [10]:
# Set the number of samples and parameters
num_samples = 100
num_parameters = 9

# Define the ranges for each parameter
parameter_bounds = [
    [0.25, 0.25],    # Range for parameter 1 mu_max
    [15, 25],       # Range for parameter 2 X_max
    [0.1, 20.0],    # Range for parameter 3 - Ks
    [0.1, 20.0],    # Range for parameter 4 - Ks_qs
    [0.1, 20.0],     # Range for parameter 5 - Ki
    [0.3, 0.5],     # Range for parameter 6 - Yxs
    [0.1, 1.5],     # Range for parameter 7 - qs_max
    [0.0, 0.2],   # Range for parameter 8 - m_s
    [0.001, 1.0],  # Range for parameter 9 - lag
]

In [11]:
LHS_samples = get_LHS_samples(num_samples, num_parameters, parameter_bounds)
LHS_samples.shape

(100, 9)

##### 5. Run it

In [12]:
df_all_sets = pd.DataFrame(columns=['set', 'mu', 'qs', 'mu_max', 'X_max', 'Ks', 'Ks_qs', 'Ki', 'Yxs', 'qs_max', 'm_s', 'lag', 'rmse'])
for set_num in range(LHS_samples.shape[0]):
    # Save all parameters and equations and the RMSE in a dataframe
    ## with the beginning of one set a new rmse_overview will be created
    rmse_one_set = []
    #key = f'set{set_num}' ; init_p = est_mu_max[key]
    init_p = list(LHS_samples[set_num, :])
    for i in range(len(mu_all)):
        # Define the combination of equations
        mu_eq = mu_all[i]; num_mu = i

        # mu 0 to mu3 are combined with qs3 while for mu4 all qs except for qs3 are combined
        if i != 4:
            j = 3
            qs_eq = qs_all[j]; num_qs = j   

            # Make the predictions and calculate the error
            rmse, time_pred, biomass_pred, substrate_pred, volume_pred = objective_function(init_p, mu_eq, num_mu, qs_eq, num_qs)
            # save the parameters in a dataframe
            append_list=[set_num, i, j, init_p[0], init_p[1], init_p[2], init_p[3], init_p[4], init_p[5], init_p[6], init_p[7], init_p[8], round(rmse, 3)]
            rmse_one_set.append(append_list)

            # Make a plot and save it
            title = f'set{set_num}/ mu{i} - qs{j} - rmse: {round(rmse, 3)}'
            plot_name = f'set{set_num}_mu{i}_qs{j}_rmse{int(rmse)}'

            if rmse <= float(6):
                plot_estimation(time_pred, biomass_pred, substrate_pred, volume_pred, title, plot_name, set_num)
        
        elif i == 4:
            for j in range(3):
                qs_eq = qs_all[j]; num_qs = j   

                # Make the predictions and calculate the error
                rmse, time_pred, biomass_pred, substrate_pred, volume_pred = objective_function(init_p, mu_eq, num_mu, qs_eq, num_qs)
                # save the parameters in a dataframe
                append_list=[set_num, i, j, init_p[0], init_p[1], init_p[2], init_p[3], init_p[4], init_p[5], init_p[6], init_p[7], init_p[8], round(rmse, 3)]
                rmse_one_set.append(append_list)

                # Make a plot and save it
                title = f'set{set_num}/ mu{i} - qs{j} - rmse: {round(rmse, 3)}'
                plot_name = f'set{set_num}_mu{i}_qs{j}_rmse{int(rmse)}'

                if rmse <= float(6):
                    plot_estimation(time_pred, biomass_pred, substrate_pred, volume_pred, title, plot_name, set_num)

    # save the parameters of one set
    df_1set = pd.DataFrame(rmse_one_set, columns=['set', 'mu', 'qs', 'mu_max', 'X_max', 'Ks', 'Ks_qs', 'Ki', 'Yxs', 'qs_max', 'm_s', 'lag', 'rmse'])
    df_all_sets = pd.concat([df_all_sets,df_1set], axis=0, ignore_index=True)


df_all_sets.sort_values(by=['rmse'], ascending=True, inplace=True)
df_all_sets.to_csv(f'data/estimation/new_eq/data.csv')

df_all_sets.head(10)

,set,mu,qs,mu_max,X_max,Ks,Ks_qs,Ki,Yxs,qs_max,m_s,lag,rmse
8,1,1,3,0.25,22.132726,3.027250,10.994495,17.195832,0.330049,1.360724,0.186527,0.272500,5.320
71,10,1,3,0.25,24.823053,10.303964,1.041204,0.721632,0.314199,1.265538,0.169603,0.683126,5.460
225,32,1,3,0.25,24.691551,3.665002,17.716266,13.314317,0.320202,1.396845,0.193008,0.130056,5.494
533,76,1,3,0.25,21.157422,14.719385,10.059823,3.196185,0.356974,0.255111,0.197338,0.972540,5.944
85,12,1,3,0.25,22.862659,9.617166,18.384138,16.991492,0.339669,1.325941,0.167815,0.395293,5.948
624,89,1,3,0.25,22.220206,3.842982,19.725081,9.598040,0.369442,1.249460,0.181079,0.751060,6.369
1,0,1,3,0.25,22.625957,18.436806,1.972623,2.427178,0.416501,0.171372,0.195759,0.070889,6.453
344,49,1,3,0.25,22.540113,15.605187,13.632584,10.843273,0.344491,1.479278,0.162051,0.923945,6.464
491,70,1,3,0.25,24.197775,15.084468,6.873537,7.522512,0.447250,0.517230,0.184172,0.160774,6.838
78,11,1,3,0.25,20.644714,7.718666,18.489369,0.450031,0.313083,0.856420,0.159727,0.310027,6.842


In [13]:
df_LHS = pd.read_csv('data/estimation/LHS_sampling/data.csv')